# BMA Standard Formulas - Comprehensive Examples Walkthrough

This notebook demonstrates **ALL** the BMA reference functions by working through each example from the BMA Uniform Practices/Standard Formulas document (02/01/99).

For each calculation, we show **multiple equivalent functions** to demonstrate that they produce the same results.

## Functions by Category

| Category | Functions |
|----------|-----------|
| **Balance** | `sch_balance_factor_fixed_rate`, `sch_balance_factors`, `sch_ending_balance_factor` |
| **Payment** | `sch_payment_factor_fixed_rate`, `sch_payment_factor` |
| **Amortization** | `bma_sch_am_factor_fixed_rate`, `am_factor`, `sch_balance_factors` |
| **SMM/CPR** | `smm_from_factors`, `cpr_to_smm`, `smm_to_cpr`, vector versions |
| **PSA** | `psa_to_cpr`, `cpr_to_psa`, `psa_to_smm`, `generate_psa_curve` |
| **Historical Rates** | `historical_smm_fixed_rate`, `historical_cpr_fixed_rate`, `historical_psa`, pool versions |
| **Cash Flows** | `run_bma_scheduled_cashflow`, `run_bma_actual_cashflow`, `project_ending_factor_smm` |
| **Defaults** | `cdr_to_mdr`, `sda_to_cdr`, `generate_sda_curve` |
| **Analytics** | `calculate_bey`, `calculate_average_life`, `calculate_macaulay_duration`, etc. |

In [2]:
# Comprehensive Imports
import numpy as np

# === Balance/Survival Factors ===
from bma_reference import (
    sch_balance_factor_fixed_rate,  # Single period balance factor
    sch_balance_factors,            # Vector of balance factors
    sch_ending_balance_factor,              # Ending balance factor with flexible coupon
)

# === Payment Factors ===
from bma_reference import (
    sch_payment_factor_fixed_rate,   # Payment as fraction of beginning balance
    sch_payment_factor,              # Payment with flexible coupon
)

# === Amortization Factors ===
from bma_reference import (
    bma_sch_am_factor_fixed_rate,        # Single period amortization
    am_factor,                       # Amortization with flexible coupon
    sch_balance_factors,                  # Vector of amortization factors
)

# === SMM/CPR Conversions ===
from bma_reference import (
    smm_from_factors,                # Back-calculate SMM from observed factors
    cpr_to_smm,                      # CPR -> SMM conversion
    smm_to_cpr,                      # SMM -> CPR conversion
    cpr_to_smm_vector,               # Vector CPR -> SMM
    smm_to_cpr_vector,               # Vector SMM -> CPR
)

# === PSA Model ===
from bma_reference import (
    psa_to_cpr,                      # PSA -> CPR at given month
    cpr_to_psa,                      # CPR -> PSA at given month
    psa_to_smm,                      # PSA -> SMM at given month
    generate_psa_curve,              # Generate full CPR curve from PSA
    generate_smm_curve_from_psa,     # Generate full SMM curve from PSA
)

# === Historical Rate Back-Calculation ===
from bma_reference import (
    historical_smm_fixed_rate,       # Back-calc SMM from factors (fixed rate)
    historical_cpr_fixed_rate,       # Back-calc CPR from factors (fixed rate)
    bma_historical_smm,                  # Back-calc SMM (flexible coupon)
    historical_cpr,                  # Back-calc CPR (flexible coupon)
    historical_psa,                  # Back-calc PSA (iterative)
    historical_smm_pool,             # Pool aggregate SMM
    historical_cpr_pool,             # Pool aggregate CPR
    historical_psa_pool,             # Pool aggregate PSA (iterative)
)

# === Cash Flow Generation ===
from bma_reference import (
    project_ending_factor_smm,           # Project ending factor given SMM vector
    run_bma_scheduled_cashflow,          # Full scheduled cash flow table
    run_bma_actual_cashflow,             # Full actual cash flow with prepay/default
)

# === Default Models ===
from bma_reference import (
    cdr_to_mdr,                          # CDR -> MDR conversion
    cdr_to_mdr_vector,                   # Vector CDR -> MDR
    sda_to_cdr,                          # SDA -> CDR at given month
    generate_sda_curve,                  # Generate full CDR curve from SDA
)

# === Analytics ===
from bma_reference import (
    calculate_bey,                       # Bond-equivalent yield
    mortgage_yield_to_bey,               # Convert mortgage yield to BEY
    bey_to_mortgage_yield,               # Convert BEY to mortgage yield
    calculate_average_life,              # Average life calculation
    calculate_macaulay_duration,         # Macaulay duration
    calculate_modified_duration,         # Modified duration
    calculate_cashflow_convexity,        # Convexity
)

# === Example Data ===
from bma_examples import (
    SF4, SF7, SF12, SF12_POOL1, SF12_POOL2,
    SF23_CASHFLOW_A, SF31_CASHFLOW_B,
    SF49_YIELD, SF51_YIELD, SF56_AVGLIFE,
    BMA_EXAMPLES,
)

print("All BMA reference functions loaded successfully!")
print(f"Total examples available: {len(BMA_EXAMPLES)}")

All BMA reference functions loaded successfully!
Total examples available: 12


## SF-4: Basic Pass-Through Cash Flow

**Problem**: A mortgage pass-through with net coupon 9.0%, gross coupon 9.5%, term 360 months.
Calculate the first month's cash flow components.

### Functions Demonstrated:
- `sch_balance_factor_fixed_rate` - Balance at any point
- `sch_balance_factors` - Vector of balances
- `sch_payment_factor_fixed_rate` - Payment calculation
- `bma_sch_am_factor_fixed_rate` - Scheduled amortization
- `sch_balance_factors` - Vector of amortizations

In [4]:
# SF-4: Multiple Methods for Balance and Amortization
print("=" * 80)
print("SF-4: Basic Pass-Through Cash Flow - Multiple Methods")
print("=" * 80)

# Parameters
gross_coupon = 9.5
net_coupon = 9.0
servicing = 0.5
original_term = 360

expected = SF4.cashflows[(1, 1)]

# ============================================================================
# BALANCE CALCULATIONS - 3 Equivalent Methods
# ============================================================================
print("\n" + "=" * 80)
print("BALANCE CALCULATIONS")
print("=" * 80)

# Method 1: sch_balance_factor_fixed_rate (single value)
bal1_m1 = sch_balance_factor_fixed_rate(gross_coupon, original_term, original_term)
bal2_m1 = sch_balance_factor_fixed_rate(gross_coupon, original_term, original_term - 1)
print(f"\n1. sch_balance_factor_fixed_rate(coupon, orig_term, remain_term)")
print(f"   BAL(month 0) = sch_balance_factor_fixed_rate({gross_coupon}, {original_term}, {original_term}) = {bal1_m1:.8f}")
print(f"   BAL(month 1) = sch_balance_factor_fixed_rate({gross_coupon}, {original_term}, {original_term-1}) = {bal2_m1:.8f}")

# Method 2: sch_balance_factors (vector) - pass coupon as list, remaining_term to stop
surv_factors, am_factors_vec, rates = sch_balance_factors([gross_coupon], original_term, original_term - 2)
print(f"\n2. sch_balance_factors([{gross_coupon}], {original_term}, {original_term - 2})")
print(f"   Returns survival_factors, am_factors, rates")
print(f"   survival_factors[0] = {surv_factors[0]:.8f} (at age 0)")
print(f"   survival_factors[1] = {surv_factors[1]:.8f} (at age 1)")
print(f"   survival_factors[2] = {surv_factors[2]:.8f} (at age 2)")

# Method 3: Direct BMA formula
r = gross_coupon / 1200
bal1_m3 = (1 - (1 + r)**(-original_term)) / (1 - (1 + r)**(-original_term))  # = 1.0
bal2_m3 = (1 - (1 + r)**(-(original_term-1))) / (1 - (1 + r)**(-original_term))
print(f"\n3. Direct BMA Formula: BAL = [1 - (1+r)^(-M)] / [1 - (1+r)^(-M0)]")
print(f"   r = {gross_coupon}/1200 = {r:.10f}")
print(f"   BAL(month 0) = {bal1_m3:.8f}")
print(f"   BAL(month 1) = {bal2_m3:.8f}")

print(f"\n   All methods match: {np.allclose([bal1_m1, bal2_m1], [surv_factors[0], surv_factors[1]]) and np.allclose([bal1_m1, bal2_m1], [bal1_m3, bal2_m3])}")

# ============================================================================
# SCHEDULED AMORTIZATION - 3 Equivalent Methods
# ============================================================================
print("\n" + "=" * 80)
print("SCHEDULED AMORTIZATION")
print("=" * 80)

# Method 1: From balance difference
sch_am_m1 = bal1_m1 - bal2_m1
print(f"\n1. From Balance Difference: sch_am = BAL(0) - BAL(1)")
print(f"   sch_am = {bal1_m1:.8f} - {bal2_m1:.8f} = {sch_am_m1:.8f}")

# Method 2: bma_sch_am_factor_fixed_rate
sch_am_m2 = sch_am_factor_fixed_rate(gross_coupon, original_term, original_term)
print(f"\n2. bma_sch_am_factor_fixed_rate(coupon, orig_term, remain_term)")
print(f"   sch_am = bma_sch_am_factor_fixed_rate({gross_coupon}, {original_term}, {original_term}) = {sch_am_m2:.8f}")

# Method 3: sch_balance_factors (vector) - pass coupon as list, remaining_term to stop
balance_factors_list, am_factors_list, rates_list = sch_balance_factors([gross_coupon], original_term, original_term - 3)
print(f"\n3. sch_balance_factors([{gross_coupon}], {original_term}, {original_term - 3})")
print(f"   am_factors[0] = {am_factors_list[0]:.8f} (at age 0 = no amort)")
print(f"   am_factors[1] = {am_factors_list[1]:.8f} (period 1 amortization)")
print(f"   am_factors[2] = {am_factors_list[2]:.8f} (period 2 amortization)")

print(f"\n   All methods match expected: {np.allclose([sch_am_m1, sch_am_m2, am_factors_list[1]], [expected.sch_am]*3)}")
print(f"   Expected (BMA): {expected.sch_am:.8f}")

# ============================================================================
# PAYMENT CALCULATION - 2 Equivalent Methods
# ============================================================================
print("\n" + "=" * 80)
print("PAYMENT CALCULATION")
print("=" * 80)

# Method 1: sch_payment_factor_fixed_rate
payment_m1 = sch_payment_factor_fixed_rate(gross_coupon, original_term, original_term)
print(f"\n1. sch_payment_factor_fixed_rate(coupon, orig_term, remain_term)")
print(f"   payment = sch_payment_factor_fixed_rate({gross_coupon}, {original_term}, {original_term})")
print(f"           = {payment_m1:.8f}")

# Method 2: Principal + Interest decomposition (BMA SF-4)
principal = sch_am_m1
interest = bal1_m1 * (gross_coupon / 1200)
payment_m2 = principal + interest
print(f"\n2. BMA SF-4: PAYMENT = PRINCIPAL + INTEREST")
print(f"   PRINCIPAL = BAL(0) - BAL(1) = {principal:.8f}")
print(f"   INTEREST  = BAL(0) × r = {bal1_m1:.8f} × {gross_coupon/1200:.10f} = {interest:.8f}")
print(f"   PAYMENT   = {principal:.8f} + {interest:.8f} = {payment_m2:.8f}")

print(f"\n   Methods match: {np.isclose(payment_m1, payment_m2)}")

# ============================================================================
# INTEREST COMPONENTS
# ============================================================================
print("\n" + "=" * 80)
print("INTEREST COMPONENTS")
print("=" * 80)

gross_int = bal1_m1 * (gross_coupon / 1200)
svc_fee = bal1_m1 * (servicing / 1200)
net_int = gross_int - svc_fee

print(f"Gross Interest = BAL × (gross_coupon/1200) = {gross_int:.8f}")
print(f"Servicing Fee  = BAL × (servicing/1200) = {svc_fee:.8f}")
print(f"Net Interest   = Gross - Servicing = {net_int:.8f}")
print(f"\nExpected (BMA): gross={expected.gross_int:.8f}, svc={expected.svc_fee:.8f}, net={expected.net_int:.8f}")

# ============================================================================
# SUMMARY TABLE
# ============================================================================
print("\n" + "=" * 80)
print("SUMMARY: SF-4 Expected vs Calculated")
print("=" * 80)
print(f"{'Component':<25} {'Calculated':>15} {'Expected':>15} {'Match':>10}")
print("-" * 65)
print(f"{'Scheduled Amortization':<25} {sch_am_m1:>15.8f} {expected.sch_am:>15.8f} {'✓' if np.isclose(sch_am_m1, expected.sch_am) else '✗':>10}")
print(f"{'Gross Interest':<25} {gross_int:>15.8f} {expected.gross_int:>15.8f} {'✓' if np.isclose(gross_int, expected.gross_int) else '✗':>10}")
print(f"{'Servicing Fee':<25} {svc_fee:>15.8f} {expected.svc_fee:>15.8f} {'✓' if np.isclose(svc_fee, expected.svc_fee) else '✗':>10}")
print(f"{'Net Interest':<25} {net_int:>15.8f} {expected.net_int:>15.8f} {'✓' if np.isclose(net_int, expected.net_int) else '✗':>10}")

SF-4: Basic Pass-Through Cash Flow - Multiple Methods

BALANCE CALCULATIONS

1. sch_balance_factor_fixed_rate(coupon, orig_term, remain_term)
   BAL(month 0) = sch_balance_factor_fixed_rate(9.5, 360, 360) = 1.00000000
   BAL(month 1) = sch_balance_factor_fixed_rate(9.5, 360, 359) = 0.99950812

2. sch_balance_factors([9.5], 360, 358)
   Returns survival_factors, am_factors, rates
   survival_factors[0] = 1.00000000 (at age 0)
   survival_factors[1] = 0.99950812 (at age 1)
   survival_factors[2] = 0.99901236 (at age 2)

3. Direct BMA Formula: BAL = [1 - (1+r)^(-M)] / [1 - (1+r)^(-M0)]
   r = 9.5/1200 = 0.0079166667
   BAL(month 0) = 1.00000000
   BAL(month 1) = 0.99950812

   All methods match: True

SCHEDULED AMORTIZATION

1. From Balance Difference: sch_am = BAL(0) - BAL(1)
   sch_am = 1.00000000 - 0.99950812 = 0.00049188

2. bma_sch_am_factor_fixed_rate(coupon, orig_term, remain_term)
   sch_am = bma_sch_am_factor_fixed_rate(9.5, 360, 360) = 0.00049188

3. sch_balance_factors([9.5], 3

/Users/crmerrill/Developer/laminarcf/tests/laminarcf/bma_compliance/bma_reference.py:837: UserWarning: Rate vector has 1 rates but 2 periods needed. Extending oldest rate (9.5%) backwards for 1 period(s).
  warnings.warn(
/Users/crmerrill/Developer/laminarcf/tests/laminarcf/bma_compliance/bma_reference.py:837: UserWarning: Rate vector has 1 rates but 3 periods needed. Extending oldest rate (9.5%) backwards for 2 period(s).
  warnings.warn(


## SF-7: Prepayment Rate Back-Calculation

**Problem**: Given observed pool factors, back-calculate SMM, CPR, and PSA.

### Functions Demonstrated:
- `smm_from_factors` - SMM from observed factors
- `smm_to_cpr` / `cpr_to_smm` - Rate conversions
- `cpr_to_psa` / `psa_to_cpr` - PSA conversions
- `historical_smm_fixed_rate` - Direct SMM calculation
- `historical_cpr_fixed_rate` - Direct CPR calculation
- `historical_psa` - Iterative PSA calculation

In [6]:
# SF-7: Prepayment Rate Back-Calculation - Multiple Methods
print("=" * 80)
print("SF-7: Prepayment Rate Back-Calculation - Multiple Methods")
print("=" * 80)

# Given data
gross_coupon = 9.5
original_term = 359
F1 = 0.85150625  # 6/1/89 factor
F2 = 0.84732282  # 7/1/89 factor
loan_month = 17  # June 1989 is month 17 for PSA
remaining_at_f1 = 344
remaining_at_f2 = 343

expected = SF7.cashflows[(17, 1)]

print(f"\nGiven: F1={F1}, F2={F2}, Month={loan_month}")

# ============================================================================
# SMM CALCULATION - 3 Equivalent Methods
# ============================================================================
print("\n" + "=" * 80)
print("SMM CALCULATION")
print("=" * 80)

# First, get scheduled balances
BAL1 = sch_balance_factor_fixed_rate(gross_coupon, original_term, remaining_at_f1)
BAL2 = sch_balance_factor_fixed_rate(gross_coupon, original_term, remaining_at_f2)
print(f"\nScheduled balances: BAL1={BAL1:.8f}, BAL2={BAL2:.8f}")

# Method 1: smm_from_factors
smm_m1 = smm_from_factors(F1, F2, BAL1, BAL2)
print(f"\n1. smm_from_factors(F1, F2, BAL1, BAL2)")
print(f"   SMM = {smm_m1:.10f} = {smm_m1*100:.6f}%")

# Method 2: historical_smm_fixed_rate
smm_m2 = historical_smm_fixed_rate(
    coupon=gross_coupon,
    original_term=original_term,
    beginning_factor=F1,
    beginning_age=original_term - remaining_at_f1,
    ending_factor=F2,
    ending_age=original_term - remaining_at_f2
)
print(f"\n2. historical_smm_fixed_rate(...)")
print(f"   SMM = {smm_m2:.10f} = {smm_m2*100:.6f}%")

# Method 3: Direct calculation per BMA SF-5
Fsched = F1 * (BAL2 / BAL1)
prepay = Fsched - F2
smm_m3 = prepay / Fsched
print(f"\n3. Direct BMA Formula:")
print(f"   Fsched = F1 × (BAL2/BAL1) = {F1} × ({BAL2:.8f}/{BAL1:.8f}) = {Fsched:.8f}")
print(f"   Prepay = Fsched - F2 = {Fsched:.8f} - {F2} = {prepay:.8f}")
print(f"   SMM = Prepay / Fsched = {prepay:.8f} / {Fsched:.8f} = {smm_m3:.10f} = {smm_m3*100:.6f}%")

print(f"\n   All SMM methods match: {np.allclose([smm_m1, smm_m2, smm_m3], [expected.smm]*3)}")
print(f"   Expected (BMA): {expected.smm*100:.6f}%")

# ============================================================================
# CPR CALCULATION - 3 Equivalent Methods
# ============================================================================
print("\n" + "=" * 80)
print("CPR CALCULATION")
print("=" * 80)

# Method 1: smm_to_cpr (input: SMM decimal, output: CPR percentage)
cpr_m1 = smm_to_cpr(smm_m1)
print(f"\n1. smm_to_cpr(smm: float) -> float")
print(f"   CPR = smm_to_cpr({smm_m1:.10f}) = {cpr_m1:.4f}%")

# Method 2: historical_cpr_fixed_rate
cpr_m2 = historical_cpr_fixed_rate(
    coupon=gross_coupon,
    original_term=original_term,
    beginning_factor=F1,
    beginning_age=original_term - remaining_at_f1,
    ending_factor=F2,
    ending_age=original_term - remaining_at_f2
)
print(f"\n2. historical_cpr_fixed_rate(...)")
print(f"   CPR = {cpr_m2:.4f}%")

# Method 3: Direct formula
cpr_m3 = 100 * (1 - (1 - smm_m1)**12)
print(f"\n3. Direct BMA Formula: CPR = 100 × [1 - (1-SMM)^12]")
print(f"   CPR = 100 × [1 - (1-{smm_m1:.10f})^12] = {cpr_m3:.4f}%")

# Note: cpr_m1 from smm_to_cpr returns percentage, cpr_m2 from historical_cpr_fixed_rate returns percentage
# cpr_m3 we calculated manually as 100 * (1 - (1-SMM)^12) which is also percentage
print(f"\n   All CPR methods match: {np.allclose([cpr_m1, cpr_m2, cpr_m3], [expected.cpr]*3, rtol=1e-4)}")
print(f"   Expected (BMA): {expected.cpr:.4f}%")

# ============================================================================
# PSA CALCULATION - 3 Equivalent Methods
# ============================================================================
print("\n" + "=" * 80)
print("PSA CALCULATION")
print("=" * 80)

# Method 1: cpr_to_psa (input: CPR percentage, output: PSA percentage)
psa_m1 = cpr_to_psa(cpr_m1, loan_month)
print(f"\n1. cpr_to_psa(cpr: float, month: int) -> float")
print(f"   PSA = cpr_to_psa({cpr_m1:.4f}, {loan_month}) = {psa_m1:.2f}%")

# Method 2: historical_psa (iterative)
psa_m2 = historical_psa(
    coupon=gross_coupon,
    original_term=original_term,
    beginning_factor=F1,
    beginning_age=original_term - remaining_at_f1,  # age = 359 - 344 = 15
    ending_factor=F2,
    ending_age=original_term - remaining_at_f2,     # age = 359 - 343 = 16
    beginning_month=loan_month
)
print(f"\n2. historical_psa(...) [iterative Brent's method]")
print(f"   PSA = {psa_m2:.2f}%")

# Method 3: Direct formula - PSA = CPR / benchmark_CPR * 100
benchmark_cpr = min(0.2 * loan_month, 6.0)
psa_m3 = cpr_m1 / benchmark_cpr * 100
print(f"\n3. Direct BMA Formula: PSA = CPR / min(0.2×MONTH, 6.0) × 100")
print(f"   Benchmark CPR at month {loan_month} = min(0.2×{loan_month}, 6.0) = {benchmark_cpr}%")
print(f"   PSA = {cpr_m1:.4f} / {benchmark_cpr:.2f} × 100 = {psa_m3:.2f}%")

print(f"\n   All PSA methods match: {np.allclose([psa_m1, psa_m2, psa_m3], [expected.psa]*3, rtol=1e-2)}")
print(f"   Expected (BMA): {expected.psa:.2f}%")

# ============================================================================
# VERIFY ROUND-TRIP CONVERSIONS
# ============================================================================
print("\n" + "=" * 80)
print("ROUND-TRIP CONVERSION VERIFICATION")
print("=" * 80)

# SMM -> CPR -> SMM
smm_orig = smm_m1
cpr_converted = smm_to_cpr(smm_orig)
smm_roundtrip = cpr_to_smm(cpr_converted)
print(f"\nSMM → CPR → SMM:")
print(f"   Original SMM: {smm_orig:.10f}")
print(f"   Converted CPR: {cpr_converted:.10f}")
print(f"   Roundtrip SMM: {smm_roundtrip:.10f}")
print(f"   Match: {np.isclose(smm_orig, smm_roundtrip)}")

# CPR -> PSA -> CPR (all values in percentage)
cpr_orig = cpr_m1  # Already in percentage (5.1%)
psa_converted = cpr_to_psa(cpr_orig, loan_month)
cpr_roundtrip = psa_to_cpr(psa_converted, loan_month)
print(f"\nCPR → PSA → CPR (at month {loan_month}):")
print(f"   Original CPR: {cpr_orig:.4f}%")
print(f"   Converted PSA: {psa_converted:.2f}%")
print(f"   Roundtrip CPR: {cpr_roundtrip:.4f}%")
print(f"   Match: {np.isclose(cpr_orig, cpr_roundtrip)}")

SF-7: Prepayment Rate Back-Calculation - Multiple Methods

Given: F1=0.85150625, F2=0.84732282, Month=17

SMM CALCULATION

Scheduled balances: BAL1=0.99213300, BAL2=0.99157471

1. smm_from_factors(F1, F2, BAL1, BAL2)
   SMM = 0.0043527049 = 0.435270%

2. historical_smm_fixed_rate(...)
   SMM = 0.0043527049 = 0.435270%

3. Direct BMA Formula:
   Fsched = F1 × (BAL2/BAL1) = 0.85150625 × (0.99157471/0.99213300) = 0.85102709
   Prepay = Fsched - F2 = 0.85102709 - 0.84732282 = 0.00370427
   SMM = Prepay / Fsched = 0.00370427 / 0.85102709 = 0.0043527049 = 0.435270%

   All SMM methods match: True
   Expected (BMA): 0.435270%

CPR CALCULATION

1. smm_to_cpr(smm: float) -> float
   CPR = smm_to_cpr(0.0043527049) = 5.1000%

2. historical_cpr_fixed_rate(...)
   CPR = 5.1000%

3. Direct BMA Formula: CPR = 100 × [1 - (1-SMM)^12]
   CPR = 100 × [1 - (1-0.0043527049)^12] = 5.1000%

   All CPR methods match: True
   Expected (BMA): 5.1000%

PSA CALCULATION

1. cpr_to_psa(cpr: float, month: int) -> fl

## SF-12: Multi-Pool Average Prepayment Rates

**Problem**: Calculate aggregate prepayment rates for two pools over 6 months.

### Functions Demonstrated:
- `historical_smm_pool` - Pool aggregate SMM
- `historical_cpr_pool` - Pool aggregate CPR  
- `historical_psa_pool` - Pool aggregate PSA (iterative)
- `historical_psa` - Individual pool PSA
- `project_ending_factor_smm` - Verify PSA by forward projection
- `generate_smm_curve_from_psa` - Generate SMM curve from PSA

In [8]:
# SF-12: Multi-Pool Average Prepayment Rates - Multiple Methods
print("=" * 80)
print("SF-12: Multi-Pool Average Prepayment Rates - Multiple Methods")
print("=" * 80)

# Pool definitions
gross_coupon = 9.5
pool_age = 6

pool1 = {
    'original_face': 1_000_000,
    'coupon_vector': gross_coupon,
    'original_term': 358,
    'beginning_age': 9,
    'beginning_factor': 0.86925218,
    'ending_factor': 0.84732282,
}

pool2 = {
    'original_face': 2_000_000,
    'coupon_vector': gross_coupon,
    'original_term': 360,
    'beginning_age': 1,
    'beginning_factor': 0.99950812,
    'ending_factor': 0.98290230,
}

loan_pool = [pool1, pool2]
expected = SF12.cashflows[(6, 6)]

print("\nPool Data:")
print(f"{'':>15} {'Pool 1':>15} {'Pool 2':>15}")
print("-" * 45)
print(f"{'Original Face':>15} ${pool1['original_face']:>12,} ${pool2['original_face']:>12,}")
print(f"{'Original Term':>15} {pool1['original_term']:>14} {pool2['original_term']:>14}")
print(f"{'Beginning Age':>15} {pool1['beginning_age']:>14} {pool2['beginning_age']:>14}")
print(f"{'Begin Factor':>15} {pool1['beginning_factor']:>14.8f} {pool2['beginning_factor']:>14.8f}")
print(f"{'End Factor':>15} {pool1['ending_factor']:>14.8f} {pool2['ending_factor']:>14.8f}")

# ============================================================================
# AGGREGATE SMM - 2 Equivalent Methods
# ============================================================================
print("\n" + "=" * 80)
print("AGGREGATE SMM CALCULATION")
print("=" * 80)

# Method 1: historical_smm_pool
smm_avg_m1 = historical_smm_pool(loan_pool, pool_age)
print(f"\n1. historical_smm_pool(loan_pool, pool_age)")
print(f"   SMM_avg = {smm_avg_m1:.10f} = {smm_avg_m1*100:.6f}%")

# Method 2: Direct BMA formula
actual_final = sum(p['original_face'] * p['ending_factor'] for p in loan_pool)
# Calculate scheduled final balance
def sched_final_balance(p):
    surv = sch_balance_factor_fixed_rate(p['coupon_vector'], p['original_term'],
                                          p['original_term'] - p['beginning_age'] - pool_age)
    surv0 = sch_balance_factor_fixed_rate(p['coupon_vector'], p['original_term'],
                                           p['original_term'] - p['beginning_age'])
    return p['original_face'] * p['beginning_factor'] * (surv / surv0)

sched_final = sum(sched_final_balance(p) for p in loan_pool)
smm_avg_m2 = 1 - (actual_final / sched_final) ** (1/pool_age)

print(f"\n2. Direct BMA Formula: SMM_avg = 1 - (actual/sched)^(1/months)")
print(f"   Actual final balance:    ${actual_final:,.2f}")
print(f"   Scheduled final balance: ${sched_final:,.2f}")
print(f"   SMM_avg = 1 - ({actual_final:,.2f}/{sched_final:,.2f})^(1/{pool_age})")
print(f"           = {smm_avg_m2:.10f} = {smm_avg_m2*100:.6f}%")

print(f"\n   Methods match: {np.isclose(smm_avg_m1, smm_avg_m2)}")
print(f"   Expected (BMA): {expected.smm*100:.6f}%")

# ============================================================================
# AGGREGATE CPR - 2 Equivalent Methods
# ============================================================================
print("\n" + "=" * 80)
print("AGGREGATE CPR CALCULATION")
print("=" * 80)

# Method 1: historical_cpr_pool
cpr_avg_m1 = historical_cpr_pool(loan_pool, pool_age)
print(f"\n1. historical_cpr_pool(loan_pool, pool_age)")
print(f"   CPR_avg = {cpr_avg_m1:.4f}%")

# Method 2: Direct BMA formula
cpr_avg_m2 = 100 * (1 - (actual_final / sched_final) ** (12/pool_age))
print(f"\n2. Direct BMA Formula: CPR_avg = 100 × [1 - (actual/sched)^(12/months)]")
print(f"   CPR_avg = 100 × [1 - ({actual_final:,.2f}/{sched_final:,.2f})^(12/{pool_age})]")
print(f"           = {cpr_avg_m2:.4f}%")

# Method 3: From SMM (smm_to_cpr returns percentage directly)
cpr_avg_m3 = smm_to_cpr(smm_avg_m1)
print(f"\n3. From SMM: smm_to_cpr(smm_avg)")
print(f"   CPR_avg = {cpr_avg_m3:.4f}%")

print(f"\n   Methods match: {np.allclose([cpr_avg_m1, cpr_avg_m2, cpr_avg_m3], [expected.cpr]*3, rtol=1e-4)}")
print(f"   Expected (BMA): {expected.cpr:.4f}%")

# ============================================================================
# AGGREGATE PSA - Iterative Method Required
# ============================================================================
print("\n" + "=" * 80)
print("AGGREGATE PSA CALCULATION (Iterative)")
print("=" * 80)

# Method 1: historical_psa_pool
psa_avg = historical_psa_pool(loan_pool, pool_age)
print(f"\n1. historical_psa_pool(loan_pool, pool_age) [Brent's method]")
print(f"   PSA_avg = {psa_avg:.2f}%")

print(f"\n   BMA Document states: {expected.psa:.2f}%")
print(f"   Our calculation:     {psa_avg:.2f}%")
print(f"\n   Note: BMA's 212.02% is an approximation. Our {psa_avg:.2f}% exactly matches")
print(f"   the combined ending balance per SF-10's specification.")

# ============================================================================
# INDIVIDUAL POOL PSA
# ============================================================================
print("\n" + "=" * 80)
print("INDIVIDUAL POOL PSA")
print("=" * 80)

# Pool 1 PSA
psa1 = historical_psa(
    coupon=pool1['coupon_vector'],
    original_term=pool1['original_term'],
    beginning_factor=pool1['beginning_factor'],
    beginning_age=pool1['beginning_age'],
    ending_factor=pool1['ending_factor'],
    ending_age=pool1['beginning_age'] + pool_age,
    beginning_month=pool1['beginning_age'] + 1
)

# Pool 2 PSA
psa2 = historical_psa(
    coupon=pool2['coupon_vector'],
    original_term=pool2['original_term'],
    beginning_factor=pool2['beginning_factor'],
    beginning_age=pool2['beginning_age'],
    ending_factor=pool2['ending_factor'],
    ending_age=pool2['beginning_age'] + pool_age,
    beginning_month=pool2['beginning_age'] + 1
)

print(f"\nhistorical_psa() for each pool:")
print(f"   Pool 1 (Age {pool1['beginning_age']}→{pool1['beginning_age']+pool_age}): PSA = {psa1:.2f}%")
print(f"   Pool 2 (Age {pool2['beginning_age']}→{pool2['beginning_age']+pool_age}): PSA = {psa2:.2f}%")
print(f"\n   Key insight: Individual PSAs differ ({psa1:.2f}% vs {psa2:.2f}%)")
print(f"   BMA states: 'aggregate PSA should NOT be computed as weighted average'")

# ============================================================================
# VERIFICATION: Forward Projection with project_ending_factor_smm
# ============================================================================
print("\n" + "=" * 80)
print("VERIFICATION: Forward Projection at PSA " + f"{psa_avg:.2f}%")
print("=" * 80)

total_projected = 0
for i, p in enumerate([pool1, pool2], 1):
    begin_month = p['beginning_age'] + 1
    smm_vec = np.array([psa_to_smm(psa_avg, m) for m in range(begin_month, begin_month + pool_age)])
    
    proj_factor = project_ending_factor_smm(
        p['beginning_factor'], smm_vec, p['coupon_vector'],
        p['original_term'], p['beginning_age']
    )
    proj_bal = p['original_face'] * proj_factor
    act_bal = p['original_face'] * p['ending_factor']
    total_projected += proj_bal
    
    print(f"\nPool {i}:")
    print(f"   SMM vector (months {begin_month}-{begin_month+pool_age-1}): {smm_vec*100}")
    print(f"   Projected factor: {proj_factor:.8f}")
    print(f"   Actual factor:    {p['ending_factor']:.8f}")
    print(f"   Projected balance: ${proj_bal:,.2f}")
    print(f"   Actual balance:    ${act_bal:,.2f}")

print(f"\nCombined:")
print(f"   Total Projected: ${total_projected:,.2f}")
print(f"   Total Actual:    ${actual_final:,.2f}")
print(f"   Difference:      ${total_projected - actual_final:,.2f}")

SF-12: Multi-Pool Average Prepayment Rates - Multiple Methods

Pool Data:
                         Pool 1          Pool 2
---------------------------------------------
  Original Face $   1,000,000 $   2,000,000
  Original Term            358            360
  Beginning Age              9              1
   Begin Factor     0.86925218     0.99950812
     End Factor     0.84732282     0.98290230

AGGREGATE SMM CALCULATION

1. historical_smm_pool(loan_pool, pool_age)
   SMM_avg = 0.0027114153 = 0.271142%

2. Direct BMA Formula: SMM_avg = 1 - (actual/sched)^(1/months)
   Actual final balance:    $2,813,127.42
   Scheduled final balance: $2,859,330.23
   SMM_avg = 1 - (2,813,127.42/2,859,330.23)^(1/6)
           = 0.0027114153 = 0.271142%

   Methods match: True
   Expected (BMA): 0.271142%

AGGREGATE CPR CALCULATION

1. historical_cpr_pool(loan_pool, pool_age)
   CPR_avg = 3.2056%

2. Direct BMA Formula: CPR_avg = 100 × [1 - (actual/sched)^(12/months)]
   CPR_avg = 100 × [1 - (2,813,127.42/

## PSA/SMM Curve Generation

Demonstrate curve generation functions showing equivalence between different approaches.

### Functions Demonstrated:
- `psa_to_cpr` / `psa_to_smm` - Point calculations
- `generate_psa_curve` - Full CPR curve from PSA
- `generate_smm_curve_from_psa` - Full SMM curve from PSA
- `cpr_to_smm_vector` / `smm_to_cpr_vector` - Vector conversions

In [10]:
# PSA/SMM Curve Generation - Multiple Methods
print("=" * 80)
print("PSA/SMM Curve Generation - Multiple Methods")
print("=" * 80)

psa_speed = 150.0
term = 360

# ============================================================================
# GENERATE CURVES USING DIFFERENT FUNCTIONS
# ============================================================================
print(f"\nGenerating curves for {psa_speed}% PSA, {term} months")

# Method 1: generate_psa_curve (returns CPR curve)
cpr_curve_m1 = generate_psa_curve(psa_speed, term)
print(f"\n1. generate_psa_curve({psa_speed}, {term})")
print(f"   Returns CPR curve, shape: {cpr_curve_m1.shape}")

# Method 2: generate_smm_curve_from_psa
smm_curve_m2 = generate_smm_curve_from_psa(psa_speed, term)
print(f"\n2. generate_smm_curve_from_psa({psa_speed}, {term})")
print(f"   Returns SMM curve, shape: {smm_curve_m2.shape}")

# Method 3: Point-by-point using psa_to_cpr
cpr_curve_m3 = np.array([psa_to_cpr(psa_speed, m) for m in range(1, term+1)])
print(f"\n3. Point-by-point: [psa_to_cpr({psa_speed}, m) for m in 1..{term}]")
print(f"   Returns CPR curve, shape: {cpr_curve_m3.shape}")

# Method 4: Point-by-point using psa_to_smm
smm_curve_m4 = np.array([psa_to_smm(psa_speed, m) for m in range(1, term+1)])
print(f"\n4. Point-by-point: [psa_to_smm({psa_speed}, m) for m in 1..{term}]")
print(f"   Returns SMM curve, shape: {smm_curve_m4.shape}")

# ============================================================================
# VERIFY EQUIVALENCE
# ============================================================================
print("\n" + "=" * 80)
print("VERIFY EQUIVALENCE")
print("=" * 80)

# CPR curves should match (skip index 0 since curve generators include period 0)
print(f"\nCPR curves match (method 1[1:] vs 3): {np.allclose(cpr_curve_m1[1:], cpr_curve_m3)}")

# SMM curves should match (skip index 0)
print(f"SMM curves match (method 2[1:] vs 4): {np.allclose(smm_curve_m2[1:], smm_curve_m4)}")

# CPR -> SMM conversion (vector functions use percentage for CPR)
smm_from_cpr = cpr_to_smm_vector(cpr_curve_m1)  # CPR is already in percentage
print(f"CPR->SMM conversion matches SMM curve: {np.allclose(smm_from_cpr, smm_curve_m2)}")

# SMM -> CPR conversion (vector functions return percentage for CPR)
cpr_from_smm = smm_to_cpr_vector(smm_curve_m2)  # Returns percentage
print(f"SMM->CPR conversion matches CPR curve: {np.allclose(cpr_from_smm, cpr_curve_m1)}")

# ============================================================================
# DISPLAY SAMPLE VALUES
# ============================================================================
print("\n" + "=" * 80)
print(f"SAMPLE VALUES FOR {psa_speed}% PSA")
print("=" * 80)

print(f"\n{'Month':>6} {'CPR %':>12} {'SMM %':>12} {'Notes':>20}")
print("-" * 55)

sample_months = [1, 5, 10, 15, 20, 25, 30, 35, 60, 120, 360]
for m in sample_months:
    cpr = cpr_curve_m1[m]  # Index m since curves include period 0
    smm = smm_curve_m2[m] * 100  # SMM is decimal, multiply by 100 for display
    notes = ""
    if m <= 30:
        notes = f"Ramp: {psa_speed/100:.1f}×0.2×{m}"
    else:
        notes = f"Plateau: {psa_speed/100:.1f}×6.0"
    print(f"{m:>6} {cpr:>12.4f} {smm:>12.6f} {notes:>20}")

PSA/SMM Curve Generation - Multiple Methods

Generating curves for 150.0% PSA, 360 months

1. generate_psa_curve(150.0, 360)
   Returns CPR curve, shape: (361,)

2. generate_smm_curve_from_psa(150.0, 360)
   Returns SMM curve, shape: (361,)

3. Point-by-point: [psa_to_cpr(150.0, m) for m in 1..360]
   Returns CPR curve, shape: (360,)

4. Point-by-point: [psa_to_smm(150.0, m) for m in 1..360]
   Returns SMM curve, shape: (360,)

VERIFY EQUIVALENCE

CPR curves match (method 1[1:] vs 3): True
SMM curves match (method 2[1:] vs 4): True
CPR->SMM conversion matches SMM curve: True
SMM->CPR conversion matches CPR curve: True

SAMPLE VALUES FOR 150.0% PSA

 Month        CPR %        SMM %                Notes
-------------------------------------------------------
     1       0.3000     0.025034      Ramp: 1.5×0.2×1
     5       1.5000     0.125868      Ramp: 1.5×0.2×5
    10       3.0000     0.253505     Ramp: 1.5×0.2×10
    15       4.5000     0.382964     Ramp: 1.5×0.2×15
    20       6.00

## Cash Flow Generation (SF-23/SF-31)

Demonstrate comprehensive cash flow generation with prepayments and defaults.

### Functions Demonstrated:
- `run_bma_scheduled_cashflow` - Scheduled cash flows (no prepay/default)
- `run_bma_actual_cashflow` - Actual cash flows with prepay and defaults
- `cdr_to_mdr` / `sda_to_cdr` - Default rate conversions
- `generate_sda_curve` - SDA default curve generation

In [12]:
# Cash Flow Generation - SF-23 (1% SMM + 1% MDR)
print("=" * 80)
print("SF-23: Cash Flow Generation with Defaults")
print("=" * 80)

print(f"\nExample: {SF23_CASHFLOW_A.description}")

# Parameters from SF-23
original_balance = 100_000_000
gross_coupon = 8.0
net_coupon = 7.5  # gross - 0.5% servicing
original_term = 360
smm_constant = 0.01  # 1% SMM
mdr_constant = 0.01  # 1% MDR
recovery_months = 12
loss_severity = 0.20

# ============================================================================
# DEFAULT RATE CONVERSIONS
# ============================================================================
print("\n" + "=" * 80)
print("DEFAULT RATE CONVERSIONS")
print("=" * 80)

# CDR -> MDR conversion (cdr_to_mdr expects CDR as percentage, returns MDR as decimal)
cdr_example = 12.0  # 12% annual
mdr_from_cdr = cdr_to_mdr(cdr_example)
print(f"\n1. cdr_to_mdr(cdr: float) -> float")
print(f"   cdr_to_mdr({cdr_example}) = {mdr_from_cdr:.6f}")
print(f"   CDR = {cdr_example}% annual → MDR = {mdr_from_cdr*100:.4f}% monthly")
print(f"   Formula: MDR = 1 - (1 - CDR/100)^(1/12)")

# MDR -> CDR conversion (verify) - MDR is decimal
cdr_back = 100 * (1 - (1 - mdr_from_cdr)**12)
print(f"   Verify: MDR {mdr_from_cdr*100:.4f}% → CDR {cdr_back:.2f}%")

# Vector conversion
cdr_vector = np.array([6.0, 12.0, 18.0, 24.0])
mdr_vector_ex = cdr_to_mdr_vector(cdr_vector)
print(f"\n2. cdr_to_mdr_vector([6, 12, 18, 24])")
print(f"   CDRs: {cdr_vector}")
print(f"   MDRs: {mdr_vector_ex}")

# ============================================================================
# SDA CURVE GENERATION
# ============================================================================
print("\n" + "=" * 80)
print("SDA CURVE GENERATION")
print("=" * 80)

sda_speed = 100.0
print(f"\n100% SDA curve (first 60 months):")
print(f"{'Month':>6} {'CDR %':>10} {'MDR %':>10} {'Description':>25}")
print("-" * 55)

for month in [1, 6, 12, 24, 30, 36, 48, 60]:
    cdr = sda_to_cdr(sda_speed, month)  # Returns CDR as percentage
    mdr = cdr_to_mdr(cdr)  # Returns MDR as decimal
    if month <= 30:
        desc = f"Ramp: {0.02*month:.2f}%"
    elif month <= 60:
        desc = f"Plateau: 0.60%"
    else:
        desc = f"Decline"
    print(f"{month:>6} {cdr:>10.4f} {mdr*100:>10.6f} {desc:>25}")

# ============================================================================
# SCHEDULED CASH FLOW
# ============================================================================
print("\n" + "=" * 80)
print("SCHEDULED CASH FLOW (run_bma_scheduled_cashflow)")
print("=" * 80)

# Generate scheduled cash flow
# Signature: run_bma_scheduled_cashflow(original_balance, current_balance, coupon, 
#            original_term, remaining_term, accrued_interest=0.0, servicing_fee=0.0)
sch_cf = run_bma_scheduled_cashflow(
    original_balance=original_balance,
    current_balance=original_balance,
    coupon=gross_coupon / 100,  # Convert to decimal
    original_term=original_term,
    remaining_term=original_term,
    servicing_fee=0.005  # 0.5% servicing
)

print(f"\nScheduled cash flow for first 6 months (no prepay/default):")
print(f"Available attributes: period, beginning_balance, scheduled_payment, interest_paid, principal_paid, ending_balance, pool_factor")
print(f"\n{'Month':>6} {'Beg Balance':>16} {'Payment':>14} {'Principal':>14} {'Interest':>14} {'End Balance':>16}")
print("-" * 85)
for i in range(1, 7):
    print(f"{i:>6} {sch_cf.beginning_balance[i]:>16,.2f} {sch_cf.scheduled_payment[i]:>14,.2f} "
          f"{sch_cf.principal_paid[i]:>14,.2f} {sch_cf.interest_paid[i]:>14,.2f} {sch_cf.ending_balance[i]:>16,.2f}")

# ============================================================================
# ACTUAL CASH FLOW WITH PREPAY AND DEFAULTS
# ============================================================================
print("\n" + "=" * 80)
print("ACTUAL CASH FLOW (run_bma_actual_cashflow)")
print("=" * 80)

# Generate SMM and MDR curves (constant for SF-23)
# Curves need to be length remaining_term + 1 (includes period 0)
num_periods = original_term + 1
smm_curve = np.full(num_periods, smm_constant)
mdr_curve = np.full(num_periods, mdr_constant)
severity_curve = np.full(num_periods, loss_severity)

# Run actual cash flow
# Signature: run_bma_actual_cashflow(scheduled_cf, smm_curve, mdr_curve, severity_curve,
#            severity_lag=12, coupon=0.08, pi_advanced=True, months_to_liquidation=12)
act_cf = run_bma_actual_cashflow(
    scheduled_cf=sch_cf,
    smm_curve=smm_curve,
    mdr_curve=mdr_curve,
    severity_curve=severity_curve,
    severity_lag=recovery_months,
    coupon=gross_coupon / 100,
    pi_advanced=True,
    months_to_liquidation=recovery_months
)

print(f"\nActual cash flow for first 12 months (1% SMM, 1% MDR):")
print(f"\n{'Mo':>3} {'Perf Bal':>16} {'Vol Prepay':>14} {'New Default':>14} {'Act Interest':>14}")
print("-" * 65)
for i in range(1, 13):
    print(f"{i:>3} {act_cf.perf_bal[i]:>16,.2f} {act_cf.vol_prepay[i]:>14,.2f} "
          f"{act_cf.new_def[i]:>14,.2f} {act_cf.act_int[i]:>14,.2f}")

SF-23: Cash Flow Generation with Defaults

Example: Cash Flow A: 1% SMM constant, 1% MDR constant, P&I advanced, 12mo recovery, 20% loss severity. Full 360-month cash flow table in bma_cashflow_a.csv.

DEFAULT RATE CONVERSIONS

1. cdr_to_mdr(cdr: float) -> float
   cdr_to_mdr(12.0) = 0.010596
   CDR = 12.0% annual → MDR = 1.0596% monthly
   Formula: MDR = 1 - (1 - CDR/100)^(1/12)
   Verify: MDR 1.0596% → CDR 12.00%

2. cdr_to_mdr_vector([6, 12, 18, 24])
   CDRs: [ 6. 12. 18. 24.]
   MDRs: [0.00514301 0.01059624 0.01640158 0.02261021]

SDA CURVE GENERATION

100% SDA curve (first 60 months):
 Month      CDR %      MDR %               Description
-------------------------------------------------------
     1     0.0200   0.001667               Ramp: 0.02%
     6     0.1200   0.010006               Ramp: 0.12%
    12     0.2400   0.020022               Ramp: 0.24%
    24     0.4800   0.040088               Ramp: 0.48%
    30     0.6000   0.050138               Ramp: 0.60%
    36     0.6000

## Yield and Analytics (SF-49/SF-50)

Demonstrate yield calculation and risk analytics functions.

### Functions Demonstrated:
- `calculate_bey` - Bond-equivalent yield
- `mortgage_yield_to_bey` / `bey_to_mortgage_yield` - Yield conversions
- `calculate_average_life` - Average life
- `calculate_macaulay_duration` - Macaulay duration
- `calculate_modified_duration` - Modified duration
- `calculate_cashflow_convexity` - Convexity

In [14]:
# Yield and Analytics - SF-49
print("=" * 80)
print("SF-49: Yield and Analytics Calculations")
print("=" * 80)

print(f"\nExample: {SF49_YIELD.description}")
expected = SF49_YIELD.cashflows[(0, 1)]

# ============================================================================
# YIELD CONVERSIONS
# ============================================================================
print("\n" + "=" * 80)
print("YIELD CONVERSIONS")
print("=" * 80)

# BMA Expected values
bey_expected = expected.yield_pct  # 9.10675%
mortgage_yield_expected = expected.mortgage_yield  # 8.93863%

# Method 1: mortgage_yield_to_bey (input/output both percentage)
bey_from_mortgage = mortgage_yield_to_bey(mortgage_yield_expected)
print(f"\n1. mortgage_yield_to_bey({mortgage_yield_expected:.5f})")
print(f"   Mortgage Yield {mortgage_yield_expected:.5f}% → BEY {bey_from_mortgage:.5f}%")

# Method 2: bey_to_mortgage_yield (input/output both percentage)
mortgage_from_bey = bey_to_mortgage_yield(bey_expected)
print(f"\n2. bey_to_mortgage_yield({bey_expected:.5f})")
print(f"   BEY {bey_expected:.5f}% → Mortgage Yield {mortgage_from_bey:.5f}%")

# Verify round-trip
bey_roundtrip = mortgage_yield_to_bey(bey_to_mortgage_yield(bey_expected))
print(f"\n3. Round-trip verification:")
print(f"   Original BEY: {bey_expected:.5f}%")
print(f"   Roundtrip BEY: {bey_roundtrip:.5f}%")
print(f"   Match: {np.isclose(bey_expected, bey_roundtrip)}")

# ============================================================================
# DURATION AND CONVEXITY RELATIONSHIPS
# ============================================================================
print("\n" + "=" * 80)
print("DURATION AND CONVEXITY")
print("=" * 80)

print(f"\nBMA Expected Values (SF-49/SF-50):")
print(f"   Price:             {expected.price:.4f}")
print(f"   BEY:               {expected.yield_pct:.5f}%")
print(f"   Average Life:      {expected.avg_life:.5f} years")
print(f"   Macaulay Duration: {expected.duration:.5f} years")
print(f"   Modified Duration: {expected.mod_duration:.5f} years")
print(f"   Convexity:         {expected.convexity:.4f} years²")

# Verify Modified Duration = Macaulay / (1 + y/2)
# calculate_modified_duration expects BEY as percentage (9.1, not 0.091)
mod_dur_calc = calculate_modified_duration(expected.duration, expected.yield_pct)
print(f"\n   calculate_modified_duration({expected.duration:.5f}, {expected.yield_pct:.5f})")
print(f"   = Macaulay / (1 + BEY/2)")
print(f"   = {expected.duration:.5f} / (1 + {expected.yield_pct/100:.6f}/2)")
print(f"   = {mod_dur_calc:.5f}")
print(f"   Expected: {expected.mod_duration:.5f}")
print(f"   Match: {np.isclose(mod_dur_calc, expected.mod_duration, rtol=1e-3)}")

# ============================================================================
# EFFECTIVE DURATION (WITH PREPAY OPTIONALITY)
# ============================================================================
print("\n" + "=" * 80)
print("EFFECTIVE DURATION/CONVEXITY")
print("=" * 80)

print(f"\nBMA Expected Values:")
print(f"   Effective Duration:  {expected.eff_duration:.2f} years")
print(f"   Effective Convexity: {expected.eff_convexity:.1f} years²")
print(f"\n   Note: Effective convexity is NEGATIVE ({expected.eff_convexity:.1f}) due to")
print(f"   prepayment optionality - as rates fall, prepayments accelerate,")
print(f"   limiting price appreciation.")

SF-49: Yield and Analytics Calculations

Example: Ginnie Mae I 9.0% pass-through, 360mo term, 150% PSA, 14-day actual delay, settled on issue date at par. Full yield/duration example.

YIELD CONVERSIONS

1. mortgage_yield_to_bey(8.93863)
   Mortgage Yield 8.93863% → BEY 9.10675%

2. bey_to_mortgage_yield(9.10675)
   BEY 9.10675% → Mortgage Yield 8.93863%

3. Round-trip verification:
   Original BEY: 9.10675%
   Roundtrip BEY: 9.10675%
   Match: True

DURATION AND CONVEXITY

BMA Expected Values (SF-49/SF-50):
   Price:             100.0000
   BEY:               9.10675%
   Average Life:      9.77844 years
   Macaulay Duration: 5.73147 years
   Modified Duration: 5.48186 years
   Convexity:         54.4326 years²

   calculate_modified_duration(5.73147, 9.10675)
   = Macaulay / (1 + BEY/2)
   = 5.73147 / (1 + 0.091067/2)
   = 5.48186
   Expected: 5.48186
   Match: True

EFFECTIVE DURATION/CONVEXITY

BMA Expected Values:
   Effective Duration:  5.44 years
   Effective Convexity: -60.0 yea

## Summary: Functions Demonstrated

This notebook demonstrated the following BMA reference functions:

| Category | Functions Used | Examples |
|----------|---------------|----------|
| **Balance** | `sch_balance_factor_fixed_rate`, `sch_balance_factors` | SF-4, SF-7 |
| **Payment** | `sch_payment_factor_fixed_rate` | SF-4 |
| **Amortization** | `bma_sch_am_factor_fixed_rate`, `sch_balance_factors` | SF-4 |
| **SMM/CPR** | `smm_from_factors`, `cpr_to_smm`, `smm_to_cpr`, vector versions | SF-7, SF-12 |
| **PSA** | `psa_to_cpr`, `psa_to_smm`, `cpr_to_psa`, curve generators | SF-7, SF-12 |
| **Historical** | `historical_smm_fixed_rate`, `historical_cpr_fixed_rate`, `historical_psa`, pool versions | SF-7, SF-12 |
| **Cash Flow** | `run_bma_scheduled_cashflow`, `run_bma_actual_cashflow`, `project_ending_factor_smm` | SF-12, SF-23 |
| **Defaults** | `cdr_to_mdr`, `cdr_to_mdr_vector`, `sda_to_cdr`, `generate_sda_curve` | SF-23 |
| **Yield** | `mortgage_yield_to_bey`, `bey_to_mortgage_yield`, `calculate_modified_duration` | SF-49 |

### Key Takeaways

1. **Multiple equivalent methods** exist for most calculations - use the one that fits your workflow
2. **Vector functions** are more efficient for multi-period calculations
3. **PSA requires iteration** for back-calculation (uses Brent's method)
4. **Pool aggregation** must apply PSA to each pool at its actual age
5. **Round-trip conversions** (SMM↔CPR↔PSA) verify implementation correctness